In [212]:
import requests
import csv
import pandas as pd
import json
import dateutil

rest_api_domain = "https://api-fxpractice.oanda.com"
access_token = "";
account_id = "101-004-7361006-006"
granularity = "H1"
instrument = "GBP_JPY"
number_of_candles = "10"
candles_endpoint = f'/v3/instruments/{instrument}/candles?count={number_of_candles}&granularity={granularity}'
authorization_header = f'Bearer {access_token}'
trades_endpoint = f"/v3/accounts/{account_id}/trades"

In [23]:
url = rest_api_domain + trades_endpoint
response = requests.get(url, headers={'Authorization': authorization_header})
response_json = response.json()
trades = response_json['trades']

In [41]:
trade = trades[0]
instrument = trade['instrument']
open_time = trade['openTime']
units = trade['initialUnits']
financing = trade['financing']
realized_pl = trade['realizedPL']
unrealized_pl = trade['unrealizedPL']
initial_margin_req = trade['initialMarginRequired']
margin_used = trade['marginUsed']
open_price = trade['price']
take_profit = None
stop_loss = 'Not Set'
tuple_header = "Instrument", 'Open Time', "Open Price", "Units", "P/L", "TP", "SL"
if 'takeProfitOrder' in trade:
    take_profit = trade['takeProfitOrder']['price']
if 'stopLossOrder' in trade:
    stop_loss = trade['stopLossOrder']['price']

trade_tuple = instrument, open_time, float(open_price), int(units), float(financing), float(take_profit), stop_loss
trade_tuple
tuple_header

<module 'csv' from '/anaconda3/lib/python3.6/csv.py'>

In [143]:
def read_as_pandas_df(trades_json):
    df = pd.read_json(json.dumps(trades))
    return df

In [128]:
trade = trades[0]
def parse_trade_from_response(trade):
    instrument = trade['instrument']
    open_time = trade['openTime']
    units = int(trade['initialUnits'])
    direction = "Buy" if units > 0 else "Sell"
    financing = trade['financing']
    realized_pl = trade['realizedPL']
    unrealized_pl = trade['unrealizedPL']
    initial_margin_req = trade['initialMarginRequired']
    margin_used = trade['marginUsed']
    open_price = trade['price']
    take_profit = None
    stop_loss = 'Not Set'
    if 'takeProfitOrder' in trade:
        take_profit = trade['takeProfitOrder']['price']
    if 'stopLossOrder' in trade:
        stop_loss = trade['stopLossOrder']['price']

    trade_list = [instrument, open_time, float(open_price), direction, units, unrealized_pl, float(take_profit), stop_loss, float(financing)]
    return trade_list

In [129]:
def get_csv_header():
    return [str("Instrument"), str('Open Time'), "Open Price", "Direction","Units", "P/L", "TP", "SL", "Financing"]

In [130]:
def write_csv(filename,rows_as_lists):
    with open(filename, 'w') as csvfile:
        spamwriter = csv.writer(csvfile,delimiter=',')
        for row in rows_as_lists:
            spamwriter.writerow(row)

In [136]:
all_trades = [ parse_trade_from_response(t) for t in trades]

In [132]:
all_trades = [get_csv_header()] + all_trades

In [133]:
all_trades

[['Instrument',
  'Open Time',
  'Open Price',
  'Direction',
  'Units',
  'P/L',
  'TP',
  'SL',
  'Financing'],
 ['NZD_USD',
  '2019-02-01T21:52:47.119613657Z',
  0.6895,
  'Buy',
  1200,
  '-0.1928',
  0.7,
  'Not Set',
  -0.057],
 ['NZD_USD',
  '2019-02-01T16:00:21.625682059Z',
  0.69128,
  'Buy',
  1100,
  '-1.6744',
  0.6968,
  '0.63000',
  -0.0589],
 ['AUD_USD',
  '2019-02-01T01:45:00.887031265Z',
  0.7251,
  'Buy',
  1100,
  '-0.4712',
  0.7295,
  '0.68000',
  -0.0807],
 ['USD_MXN',
  '2019-02-01T01:14:51.106653044Z',
  19.10209,
  'Sell',
  -1100,
  '-0.8822',
  19.0,
  'Not Set',
  0.2136],
 ['USD_JPY',
  '2019-01-31T22:55:00.101060766Z',
  108.817,
  'Sell',
  -1000,
  '-4.8273',
  108.3,
  'Not Set',
  -0.1982],
 ['EUR_USD',
  '2019-01-31T18:15:23.238596328Z',
  1.14499,
  'Buy',
  1200,
  '0.1284',
  1.15,
  'Not Set',
  -0.3318],
 ['GBP_USD',
  '2019-01-29T19:55:25.522656078Z',
  1.31149,
  'Buy',
  1500,
  '-4.7386',
  1.32,
  'Not Set',
  -0.5267],
 ['EUR_GBP',
  '2019-

In [135]:
write_csv("/Users/david/Desktop/trades.csv",all_trades)

In [204]:
# https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
df = read_as_pandas_df(trades)
#df['Direction'] = "Buy" if int(df['currentUnits']) > 0 else "Sell"
df['Direction'] = df.apply(lambda row: "Buy" if int(row['currentUnits']) > 0 else "Sell",axis=1)
df['currentUnits'] = df.apply(lambda row: -1*int(row['currentUnits']) if int(row['currentUnits']) < 0 else int(row['currentUnits']),axis=1)
#df['openTime'] = df['openTime'].apply(dateutil.parser.parse, dayfirst=True)
buys = df['currentUnits'][df['Direction'] == 'Buy'].sum()
sells = df['currentUnits'][df['Direction'] == 'Sell'].sum()
direction_counts = df['Direction'].value_counts()
print("Directions:")
print(direction_counts)
print("=================")
print(f"Units bought: {buys}")
print(f"Units sold: {sells}")



Directions:
Buy     14
Sell    11
Name: Direction, dtype: int64
Units bought: 18855
Units sold: 12900


In [210]:
df_grouped = df.groupby(['instrument','Direction'],as_index=False)
df2 = df_grouped.agg({'currentUnits':'sum'})
df2['perc'] = df2.apply(lambda row: 100*row['currentUnits']/buys if row['Direction'] == 'Buy' else 100*row['currentUnits']/sells,axis=1)
df2

,instrument,Direction,currentUnits,perc
0,AUD_JPY,Buy,2000,10.607266
1,AUD_USD,Buy,6600,35.003978
2,BCO_USD,Buy,55,0.291700
3,EUR_GBP,Sell,1000,7.751938
4,EUR_USD,Buy,1200,6.364360
5,EUR_USD,Sell,2000,15.503876
6,GBP_USD,Buy,1500,7.955449
7,GBP_USD,Sell,3700,28.682171
8,NZD_USD,Buy,2300,12.198356
9,USD_CAD,Buy,1000,5.303633
